In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import copy
import math

In [2]:
file = files.upload()

KeyboardInterrupt: 

In [3]:
df = pd.read_csv("/content/NLPdataset.csv")

In [4]:
df.head()

,sentence,part of speech
0,Pierre Vinken 61 years old will join the board...,"NNP,NNP,CD,NNS,JJ,MD,VB,DT,NN,IN,DT,JJ,NN,NNP,CD"
1,Mr Vinken is chairman of Elsevier NV the Dutch...,"NNP,NNP,VBZ,NN,IN,NNP,NNP,DT,NNP,VBG,NN"
2,Rudolph Agnew 55 years old and former chairman...,"NNP,NNP,CD,NNS,JJ,CC,JJ,NN,IN,NNP,NNP,NNP,NNP,..."
3,A form of asbestos once used to make Kent ciga...,"DT,NN,IN,NN,RB,VBN,TO,VB,NNP,NN,NNS,VBZ,VBN,DT..."
4,The asbestos fiber crocidolite is unusually re...,"DT,NN,NN,NN,VBZ,RB,JJ,IN,PRP,VBZ,DT,NNS,IN,RB,..."


In [5]:
start = int(len(df)*0.8)


X_train = df.iloc[:start,0]
X_test = df.iloc[start:,0]
y_train = df.iloc[:start,1]
y_test = df.iloc[start:,1]
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0],df.iloc[:,1], test_size=0.20, random_state=42,shuffle=True)

In [6]:
X_train

0       Pierre Vinken 61 years old will join the board...
1       Mr Vinken is chairman of Elsevier NV the Dutch...
2       Rudolph Agnew 55 years old and former chairman...
3       A form of asbestos once used to make Kent ciga...
4       The asbestos fiber crocidolite is unusually re...
                              ...                        
3126    The Treasury said the refunding is contingent ...
3127    Until such action takes places the Treasury ha...
3128    Meanwhile Treasury bonds ended modestly higher...
3129    The benchmark 30year bond about 1/4 point or 2...
3130    The benchmark was priced at 102 22/32 to yield...
Name: sentence, Length: 3131, dtype: object

In [7]:
y_train

0        NNP,NNP,CD,NNS,JJ,MD,VB,DT,NN,IN,DT,JJ,NN,NNP,CD
1                 NNP,NNP,VBZ,NN,IN,NNP,NNP,DT,NNP,VBG,NN
2       NNP,NNP,CD,NNS,JJ,CC,JJ,NN,IN,NNP,NNP,NNP,NNP,...
3       DT,NN,IN,NN,RB,VBN,TO,VB,NNP,NN,NNS,VBZ,VBN,DT...
4       DT,NN,NN,NN,VBZ,RB,JJ,IN,PRP,VBZ,DT,NNS,IN,RB,...
                              ...                        
3126    DT,NNP,VBD,DT,NN,VBZ,NN,IN,JJ,CC,JJ,NN,IN,DT,N...
3127    IN,JJ,NN,VBZ,NNS,DT,NNP,VBZ,DT,NN,TO,VB,JJ,NN,...
3128                       RB,NNP,NNS,VBD,RB,JJR,IN,JJ,NN
3129            DT,NN,JJ,NN,IN,CD,NN,CC,CD,IN,DT,CD,NN,NN
3130    DT,NN,VBD,VBN,IN,CD,CD,TO,VB,CD,VBN,IN,CD,CD,T...
Name: part of speech, Length: 3131, dtype: object

#Extracting distinct pos tags from the dataset

In [9]:
distinct_tags = set()
i = 0
for i in y_train:
    for j in (i.split(",")):
        distinct_tags.add(j)

pos_tags = {}
j = 0
for i in distinct_tags:
    pos_tags[i] = j
    j = j + 1

print(pos_tags)

{'RB': 0, 'DT': 1, 'WP$': 2, 'LS': 3, 'JJR': 4, 'EX': 5, 'NN': 6, 'PRP$': 7, 'TO': 8, 'MD': 9, 'CC': 10, 'RBS': 11, 'VBZ': 12, 'NNS': 13, 'JJS': 14, 'FW': 15, 'PDT': 16, 'VBD': 17, 'VBG': 18, 'VBP': 19, 'VBN': 20, 'WRB': 21, 'NNPS': 22, 'IN': 23, 'NNP': 24, 'JJ': 25, 'RBR': 26, 'WP': 27, 'PRP': 28, 'RP': 29, 'SYM': 30, 'VB': 31, 'CD': 32, 'WDT': 33, 'UH': 34}


In [12]:
print(y_train)

0        NNP,NNP,CD,NNS,JJ,MD,VB,DT,NN,IN,DT,JJ,NN,NNP,CD
1                 NNP,NNP,VBZ,NN,IN,NNP,NNP,DT,NNP,VBG,NN
2       NNP,NNP,CD,NNS,JJ,CC,JJ,NN,IN,NNP,NNP,NNP,NNP,...
3       DT,NN,IN,NN,RB,VBN,TO,VB,NNP,NN,NNS,VBZ,VBN,DT...
4       DT,NN,NN,NN,VBZ,RB,JJ,IN,PRP,VBZ,DT,NNS,IN,RB,...
                              ...                        
3126    DT,NNP,VBD,DT,NN,VBZ,NN,IN,JJ,CC,JJ,NN,IN,DT,N...
3127    IN,JJ,NN,VBZ,NNS,DT,NNP,VBZ,DT,NN,TO,VB,JJ,NN,...
3128                       RB,NNP,NNS,VBD,RB,JJR,IN,JJ,NN
3129            DT,NN,JJ,NN,IN,CD,NN,CC,CD,IN,DT,CD,NN,NN
3130    DT,NN,VBD,VBN,IN,CD,CD,TO,VB,CD,VBN,IN,CD,CD,T...
Name: part of speech, Length: 3131, dtype: object


#Creating transition matrix

In [10]:
count_of_distinct_pos_tags = len(pos_tags)
#declaring transition matrix
transition_count = []
for i in range(0,count_of_distinct_pos_tags):
    t = []
    for j in range(0,count_of_distinct_pos_tags):
        t.append(float(0))
    transition_count.append(t)

count_of_each_pos_tags = [0]*count_of_distinct_pos_tags
pos_tags_as_initial_count = [0]*count_of_distinct_pos_tags
count  = 0
for i in y_train:
    line = list(i.split(","))
    pos_tags_as_initial_count[pos_tags[line[0]]]+=1
    count_of_each_pos_tags[pos_tags[line[0]]]+=1
    length  = int(len(line))
    for j in range(1,length):
        count_of_each_pos_tags[pos_tags[line[j]]]+=1
        transition_count[pos_tags[line[j-1]]][pos_tags[line[j]]] += 1

In [11]:
print(transition_count)

[[155.0, 154.0, 0.0, 0.0, 37.0, 5.0, 41.0, 2.0, 54.0, 22.0, 39.0, 0.0, 101.0, 27.0, 1.0, 0.0, 1.0, 152.0, 84.0, 87.0, 180.0, 8.0, 1.0, 310.0, 35.0, 215.0, 23.0, 3.0, 32.0, 0.0, 0.0, 104.0, 81.0, 6.0, 0.0], [63.0, 16.0, 0.0, 0.0, 42.0, 1.0, 3064.0, 1.0, 3.0, 11.0, 5.0, 17.0, 32.0, 558.0, 62.0, 1.0, 0.0, 10.0, 76.0, 6.0, 63.0, 0.0, 21.0, 77.0, 833.0, 1358.0, 15.0, 5.0, 10.0, 0.0, 0.0, 0.0, 196.0, 2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0], [1.0, 8.0, 0.0, 0.0, 0.0, 1.0, 61.0, 0.0, 8.0, 0.0, 15.0, 0.0, 0.0, 65.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0, 96.0, 4.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0], [3.0, 13.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0

#Calculating initial state probability

In [13]:
initial_state_probability = [0]*count_of_distinct_pos_tags
for i in range(count_of_distinct_pos_tags):
    initial_state_probability[i] = (pos_tags_as_initial_count[i] + 1)/count_of_each_pos_tags[i]

#Calculating transition probability

In [ ]:
transition_probability = copy.deepcopy(transition_count)

for i in range(0,count_of_distinct_pos_tags ):
    for j in range(0,count_of_distinct_pos_tags ):
        transition_probability[i][j] = ((transition_probability[i][j] + 1)*(100))/(count_of_each_pos_tags[j] + 1)

print(transition_probability)

[[1.1049723756906078, 0.25, 0.43459365493263796, 4.545454545454546, 0.06688963210702341, 0.31007751937984496, 1.3888888888888888, 0.32679738562091504, 0.04213187276174426, 50.0, 0.86716594402838, 0.9735744089012517, 0.22767518340500886, 0.546448087431694, 1.2195121951219512, 20.0, 0.07908264136022143, 0.29308323563892147, 0.9523809523809523, 0.11068625477963373, 0.17482517482517482, 7.142857142857143, 0.04861448711716091, 0.07617306520414381, 0.8771929824561403, 0.17825311942959002, 9.090909090909092, 0.6578947368421053, 0.0812842918106076, 25.0, 3.225806451612903, 0.6896551724137931, 0.8057296329453895, 0.7770762506070908, 0.14485755673587639], [1.1049723756906078, 0.5, 0.1738374619730552, 4.545454545454546, 2.94314381270903, 5.5813953488372094, 1.8518518518518519, 6.5359477124183005, 1.9380661470402358, 50.0, 0.45986072789383786, 0.4172461752433936, 2.074373893245636, 13.66120218579235, 1.2195121951219512, 20.0, 1.265322261763543, 0.23446658851113716, 2.2222222222222223, 1.7106057556

#Extracting distinct words from the dataset

In [ ]:
distinct_words = set()
i = 0
for i in X_train:
    for j in (i.split(" ")):
        distinct_words.add(j)

words = {}
j = 0
for i in distinct_words:
    words[i] = j
    j = j + 1
print(words)
print(len(words))

{'': 0, 'polyproplene': 1, '53': 2, 'weekends': 3, 'superior': 4, 'Net': 5, 'Westminster': 6, 'avoid': 7, 'Shortly': 8, 'USSR': 9, 'traditional': 10, '621': 11, 'snow': 12, 'commanded': 13, 'investor': 14, 'Barclays': 15, 'maintain': 16, 'cleanliness': 17, 'craze': 18, 'begot': 19, 'Nature': 20, 'disapproval': 21, 'Wilton': 22, 'manufacture': 23, 'Kingdom': 24, 'decides': 25, 'Ogilvy': 26, "France's": 27, 'spaces': 28, 'meetings': 29, "We're": 30, 'extremely': 31, 'Mazda': 32, 'phony': 33, 'margins': 34, 'retractable': 35, 'driving': 36, 'sparking': 37, 'meaningful': 38, 'nightmare': 39, 'dairy': 40, 'East': 41, 'bled': 42, 'Stena': 43, 'cautious': 44, 'plot': 45, 'data': 46, 'viewpoints': 47, '14821;': 48, 'hardware': 49, 'mechanical': 50, "show's": 51, 'disregard': 52, "UK's": 53, 'patented': 54, 'conduct': 55, 'then': 56, 'redeem': 57, 'regimented': 58, 'Why': 59, 'year?': 60, 'escape': 61, 'Assistant': 62, 'carrier': 63, 'Candlestick': 64, 'businessmen': 65, 'queuing': 66, 'in': 67

#Creating emission matrix

In [ ]:
count_of_distinct_words = len(words)

#declaring emission count matrix
emission_count = []
for i in range(0,count_of_distinct_words):
    t = []
    for j in range(0,count_of_distinct_pos_tags):
        t.append(float(0))
    emission_count.append(t)



for i in range(0,len(X_train)):
    linex = X_train[i].split(" ")
    liney = y_train[i].split(",")
    length = len(linex)

    for j in range(0,length):
        emission_count[words[linex[j]]][pos_tags[liney[j]]] += 1

#Printing emission matrix

In [ ]:
# for i in range(0,count_of_distinct_words):
# print(emission_count)

#Calculating emission probability

In [ ]:
emission_probability = copy.deepcopy(emission_count)

for i in range(0,count_of_distinct_pos_tags):
    for j in range(0,count_of_distinct_pos_tags ):
        emission_probability[i][j] = ((emission_probability[i][j] + 1)*(100))/(count_of_each_pos_tags[j])

print(emission_probability[0])

[0.5555555555555556, 0.08340283569641367, 0.043478260869565216, 4.761904761904762, 0.06693440428380187, 0.15527950310559005, 0.46511627906976744, 0.32786885245901637, 0.02107037505267594, 100.0, 0.013140604467805518, 0.1392757660167131, 0.01265022137887413, 0.5494505494505495, 1.2345679012345678, 25.0, 0.03955696202531646, 0.05865102639296188, 0.3184713375796178, 0.010063399416322834, 0.05830903790087463, 7.6923076923076925, 0.048638132295719845, 0.015236934328813043, 0.2932551319648094, 0.059453032104637336, 10.0, 0.6622516556291391, 0.02032520325203252, 33.333333333333336, 3.3333333333333335, 0.6944444444444444, 0.08960573476702509, 10.787172011661808, 0.04830917874396135]


#Finding most frequent pos tag

In [ ]:
for i in range(0,count_of_distinct_pos_tags):
    if(count_of_each_pos_tags[i]==max(count_of_each_pos_tags)):
        most_frequent_tag_index = i
        break
print(most_frequent_tag_index)

for i in pos_tags:
    if(pos_tags[i]==most_frequent_tag_index):
        most_frequent_tag = i
print(most_frequent_tag)

19
NN


#Viterbi Algorithm

In [ ]:
def viterbiAlgorithm(transition_probability,emission_probability,initial_state_probability,count_of_each_pos_tags,pos_tags,words,most_frequent_tag,testX):
    count_of_distinct_pos_tags = len(pos_tags)


    #declaring matrix to calculate probability at each k
    prob_and_seq = []
    cursen = testX.split(" ")
    curlen = len(cursen)
    predicted_seq_of_pos_tags = []

    for i in range(0,count_of_distinct_pos_tags):
        t = []
        for j in range(0,curlen):
            t.append([0,"a","a"])
        prob_and_seq.append(t)
    #end of matrix creation

    for j in pos_tags.keys():
        if cursen[0] in words.keys():
            prob_and_seq[pos_tags[j]][0][0] = initial_state_probability[pos_tags[j]] * emission_probability[words[cursen[0]]][pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = "start"
            prob_and_seq[pos_tags[j]][0][2] = j
        else:
            prob_and_seq[pos_tags[j]][0][0] = initial_state_probability[pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = "start"
            prob_and_seq[pos_tags[j]][0][2] = j


    for i in range(1,curlen):
        if cursen[i] in words.keys():
            for j in range(0,count_of_distinct_pos_tags): #every prev state
                prev_prob = prob_and_seq[j][i-1][0]
                prev_prev_postag = prob_and_seq[j][i-1][1]
                prev_postag = prob_and_seq[j][i-1][2]

                #print(cursen[i],i,prev_postag)
                for k in pos_tags.keys():#every cur state
                    max_prob = prob_and_seq[pos_tags[k]][i][0]
                    max_postag = prob_and_seq[pos_tags[k]][i][1]

                    if(prev_prev_postag=="start"):
                        p1 = initial_state_probability[pos_tags[prev_postag]]
                    else:
                        p1 = transition_probability[pos_tags[prev_prev_postag]][pos_tags[prev_postag]]

                    p2 = transition_probability[pos_tags[prev_postag]][pos_tags[k]]
                    p3 = emission_probability[words[cursen[i]]][pos_tags[k]]

                    cur_prob = prev_prob + math.log(p1 + 1) + math.log(p2 + 1) + math.log(p3 + 1)

                    if max_prob<=cur_prob:
                            max_prob = cur_prob
                            prob_and_seq[pos_tags[k]][i][0] = max_prob
                            prob_and_seq[pos_tags[k]][i][1] = prev_postag
                            prob_and_seq[pos_tags[k]][i][2] = k

        else:
            for j in range(0,count_of_distinct_pos_tags): #every prev state
                prev_prob = prob_and_seq[j][i-1][0]
                prev_prev_postag = prob_and_seq[j][i-1][1]
                prev_postag = prob_and_seq[j][i-1][2]

                for k in pos_tags.keys():#every cur state
                    if (k==most_frequent_tag):
                        max_prob = prob_and_seq[pos_tags[k]][i][0]
                        max_postag = prob_and_seq[pos_tags[k]][i][1]

                        if(prev_prev_postag=="start"):
                            p1 = initial_state_probability[pos_tags[prev_postag]]
                        else:
                            p1 = transition_probability[pos_tags[prev_prev_postag]][pos_tags[prev_postag]]

                        p2 = transition_probability[pos_tags[prev_postag]][pos_tags[k]]

                        cur_prob = prev_prob + math.log(p1 + 1) + math.log(p2 + 1)

                        if max_prob<=cur_prob:
                                max_prob = cur_prob
                                prob_and_seq[pos_tags[k]][i][0] = max_prob
                                prob_and_seq[pos_tags[k]][i][1] = prev_postag
                                prob_and_seq[pos_tags[k]][i][2] = k
                    else:
                        prob_and_seq[pos_tags[k]][i][0] = float(0)
                        prob_and_seq[pos_tags[k]][i][1] = prev_postag
                        prob_and_seq[pos_tags[k]][i][2] = k


            #print(prob_and_seq[pos_tags[k]][i][1])

    maxiprob = prob_and_seq[0][curlen-1][0]
    prevpostag = prob_and_seq[0][curlen-1][1]
    maxipostag = prob_and_seq[0][curlen-1][2]

    for i in range(0,count_of_distinct_pos_tags):
        if prob_and_seq[i][curlen-1][0]>=maxiprob:
            maxiprob = prob_and_seq[i][curlen-1][0]
            prevpostag = prob_and_seq[i][curlen-1][1]
            maxipostag = prob_and_seq[i][curlen-1][2]


    predicted_seq_of_pos_tags.append(maxipostag)

    for i in range(curlen-2,-1,-1):
        predicted_seq_of_pos_tags.append(prevpostag)
        prevpostag = prob_and_seq[pos_tags[prevpostag]][i][1]


    predicted_seq_of_pos_tags.reverse()

    #print(predicted_seq_of_pos_tags)
    return predicted_seq_of_pos_tags

#Running Viterbi algo to predict the POS tags(36) associated with each word of a sentence

In [ ]:
predicted_y = []
for i in X_test:
    predicted_y.append(viterbiAlgorithm(transition_probability,emission_probability,initial_state_probability,count_of_each_pos_tags,pos_tags,words,most_frequent_tag,i))

#Calculating Accuracy

In [ ]:
k = 0
accuracy = []
for i in y_test:
    count = 0
    actualtags_y = i.split(",")
    curlen = len(actualtags_y)
    for j in range(0,curlen):
        if predicted_y[k][j]==actualtags_y[j]:
            count +=1
    k +=1
    accuracy.append(count/curlen)

print("Average accuracy before collapsing to 4 POS tags= ",sum(accuracy)*100/len(accuracy))

Average accuracy before collapsing to 4 POS tags=  65.09723641164095


#Applying POS tagging by collapsing 36 POS tags to 4 POS tags

#Mapping 36 Tags to 4 tags

#Creating transition matrix with 4 POS tags


In [ ]:
map_36_to_4_pos = {'NNPS': "N",'VBG': "V",'VBD': "V",'PDT': "O", 'PRP': "O", 'PRP$': "O", 'WP': "O", 'JJR': "A", 'JJ': "A", 'SYM': "O", 'NNP': "N", 'MD': "V", 'IN': "O", 'RP': "O", 'EX': "O", 'FW': "O", 'CD': "O", 'VBZ': "V", 'RBR': "A", 'NN': "N", 'TO': "O", 'LS': "O", 'VB': "V", 'DT': "O", 'WDT': "O", 'VBN': "V", 'WP$': "O", 'WRB': "O", 'NNS': "N", 'UH': "O", 'RBS': "A", 'JJS': "A", 'VBP': "V", 'CC': "O", 'RB': "A"}

In [ ]:
count_of_distinct_pos_tags = 4
new_pos_tags = {"N": 0,"V": 1,"A": 2,"O":3}

#declaring transition matrix
transition_count_4 = []
for i in range(0,count_of_distinct_pos_tags):
    t = []
    for j in range(0,count_of_distinct_pos_tags):
        t.append(float(0))
    transition_count_4.append(t)

count_of_each_pos_tags_4 = [0]*count_of_distinct_pos_tags
pos_tags_as_initial_count_4 = [0]*count_of_distinct_pos_tags
count  = 0
for i in y_train:
    line = list(i.split(","))
    pos_tags_as_initial_count_4[new_pos_tags[map_36_to_4_pos[line[0]]]]+=1
    count_of_each_pos_tags_4[new_pos_tags[map_36_to_4_pos[line[0]]]]+=1
    length  = int(len(line))
    for j in range(1,length):
        count_of_each_pos_tags_4[new_pos_tags[map_36_to_4_pos[line[j]]]]+=1
        transition_count_4[new_pos_tags[map_36_to_4_pos[line[j-1]]]][new_pos_tags[map_36_to_4_pos[line[j]]]] += 1

#Calculating initial state probability with 4 POS tags

In [ ]:
count_of_distinct_pos_tags = 4
initial_state_probability_4 = [0]*count_of_distinct_pos_tags
for i in range(count_of_distinct_pos_tags):
    initial_state_probability_4[i] = (pos_tags_as_initial_count_4[i] + 1)/count_of_each_pos_tags_4[i]

#Calculating transition probability with 4 POS tags

In [ ]:
transition_probability_4 = copy.deepcopy(transition_count_4)
count_of_distinct_pos_tags = 4
for i in range(0,count_of_distinct_pos_tags ):
    for j in range(0,count_of_distinct_pos_tags ):
        transition_probability_4[i][j] = (transition_probability_4[i][j] + 1)*100/(count_of_each_pos_tags_4[j] + 1)

print(transition_probability_4)

[[32.17944189332391, 37.64498468961678, 15.78186596583443, 33.30965315503552], [6.4111621335217235, 19.7179177878816, 19.93429697766097, 22.185541161721687], [16.94189332391381, 7.952120256100955, 12.90407358738502, 6.999582114500627], [40.71441186859767, 33.849865454208036, 47.24047306176084, 29.715837860426245]]


#Calculating emission count matrix with 4 POS tags

In [ ]:
count_of_distinct_words = len(words)
count_of_distinct_pos_tags = 4
#declaring emission count matrix with 4 POS tags
emission_count_4 = []
for i in range(0,count_of_distinct_words):
    t = []
    for j in range(0,count_of_distinct_pos_tags):
        t.append(float(0))
    emission_count_4.append(t)



for i in range(0,len(X_train)):
    linex = X_train[i].split(" ")
    liney = y_train[i].split(",")
    length = len(linex)

    for j in range(0,length):
        emission_count_4[words[linex[j]]][new_pos_tags[map_36_to_4_pos[liney[j-1]]]] += 1

#Calculating emission probability with 4 POS tags

In [ ]:
emission_probability_4 = copy.deepcopy(emission_count_4)
count_of_distinct_pos_tags = 4

for i in range(0,count_of_distinct_pos_tags):
    for j in range(0,count_of_distinct_pos_tags):
        emission_probability_4[i][j] = ((emission_probability_4[i][j] + 1)*(100))/(count_of_each_pos_tags_4[j])

print(emission_probability_4[0])

[0.6888329580076832, 0.16703786191536749, 0.3811276120383756, 0.09193865184504157]


#Finding most frequent word

In [ ]:
count_of_distinct_pos_tags = 4
for i in range(0,count_of_distinct_pos_tags):
    if(count_of_each_pos_tags_4[i]==max(count_of_each_pos_tags_4)):
        most_frequent_tag_index_4 = i
        break
print(most_frequent_tag_index_4)

for i in new_pos_tags:
    if(new_pos_tags[i]==most_frequent_tag_index_4):
        most_frequent_tag_4 = i
print(most_frequent_tag_4)

3
O


#Running Viterbi algo to predict the POS tags(4) associated with each word of a sentence

In [ ]:
predicted_y_4 = []
for i in X_test:
    predicted_y_4.append(viterbiAlgorithm(transition_probability_4,emission_probability_4,initial_state_probability_4,count_of_each_pos_tags_4,new_pos_tags,words,most_frequent_tag_4,i))

#Calculating accuracy with 4 POS tags

#Calculating overall accuracy

In [ ]:
k = 0
accuracy_4 = []
for i in y_test:
    count = 0
    actualtags_y_4 = i.split(",")
    curlen = len(actualtags_y_4)
    for j in range(0,curlen):
        if predicted_y_4[k][j]==map_36_to_4_pos[actualtags_y_4[j]]:
            count +=1
    k +=1
    accuracy_4.append(count/curlen)
print(accuracy_4)

print(sum(accuracy_4)/len(accuracy_4))

[0.21052631578947367, 0.37037037037037035, 0.5, 0.4166666666666667, 0.3076923076923077, 0.3225806451612903, 0.3125, 0.125, 0.24, 0.5, 0.32142857142857145, 0.16666666666666666, 0.29411764705882354, 0.5, 0.16666666666666666, 0.375, 0.3076923076923077, 0.18181818181818182, 0.5, 0.2777777777777778, 0.3793103448275862, 0.25, 0.25, 0.4634146341463415, 0.375, 0.34782608695652173, 0.3076923076923077, 0.0, 0.2222222222222222, 0.125, 0.3076923076923077, 0.3939393939393939, 0.0, 0.38095238095238093, 0.6111111111111112, 0.46153846153846156, 0.29411764705882354, 0.2647058823529412, 0.2777777777777778, 0.3939393939393939, 0.4444444444444444, 0.15, 0.3125, 0.4, 0.35, 0.25, 0.24, 0.2857142857142857, 0.4722222222222222, 0.3333333333333333, 0.2631578947368421, 0.4117647058823529, 0.36666666666666664, 0.21875, 0.35294117647058826, 0.2857142857142857, 0.3888888888888889, 0.23529411764705882, 0.2857142857142857, 0.36363636363636365, 0.2631578947368421, 0.21428571428571427, 0.2857142857142857, 0.28571428571

#Calculating class wise accuracy

In [ ]:
correctpredict = {"N": 0,"V": 0,"A": 0,"O":0}
totalprediction = {"N": 0,"V": 0,"A": 0,"O":0}
k = 0
accuracy_4 = []
for i in y_test:
    count = 0
    actualtags_y_4 = i.split(",")
    curlen = len(actualtags_y_4)
    for j in range(0,curlen):
        s = predicted_y_4[k][j]
        t = map_36_to_4_pos[actualtags_y_4[j]]
        if s==t:
            count += 1
            correctpredict[t] +=1
        totalprediction[t]+=1
    k +=1
    accuracy_4.append(count/curlen)

print("Average accuracy after collapsing to 4 POS tags = ",sum(accuracy_4)/len(accuracy_4))

print("\nClass wise prediction after collapsing to 4 POS tags")
for i in correctpredict:
    print(i," = ",(correctpredict[i]*100)/totalprediction[i])

Average accuracy after collapsing to 4 POS tags =  0.28390587983840415

Class wise prediction after collapsing to 4 POS tags
N  =  31.792207792207794
V  =  7.354056103108415
A  =  1.0582010582010581
O  =  42.72068511198946
